# Graphs generation

In [1]:
from sampo.generator.base import SimpleSynthetic

In [2]:
GRAPHS_TOP_BORDER = 100
GRAPHS_COUNT = 10

In [3]:
ss = SimpleSynthetic(111)

graphs = [ss.work_graph(top_border=GRAPHS_TOP_BORDER) for _ in range(GRAPHS_COUNT)]

# Metrics calculation

In [4]:
from sampo.schemas.graph import WorkGraph


def metric_vertex_count(wg: WorkGraph) -> float:
    return wg.vertex_count

def metric_min_children(wg: WorkGraph) -> float:
    return min((len(node.children) for node in wg.nodes if node.children))

def metric_max_children(wg: WorkGraph) -> float:
    return max((len(node.children) for node in wg.nodes if node.children))

def metric_min_parents(wg: WorkGraph) -> float:
    return min((len(node.parents) for node in wg.nodes if node.parents))

def metric_max_parents(wg: WorkGraph) -> float:
    return max((len(node.parents) for node in wg.nodes if node.parents))

In [5]:
dataset = [(
    metric_vertex_count(wg),
    metric_min_children(wg),
    metric_max_children(wg),
    metric_min_parents(wg),
    metric_max_parents(wg)
) for wg in graphs]

dataset

[(36, 1, 24, 1, 22),
 (150, 1, 60, 1, 42),
 (129, 1, 42, 1, 48),
 (131, 1, 40, 1, 52),
 (198, 1, 64, 1, 40),
 (129, 1, 42, 1, 58),
 (139, 1, 46, 1, 64),
 (39, 1, 27, 1, 25),
 (141, 1, 42, 1, 59),
 (135, 1, 43, 1, 58)]

In [6]:
from sampo.schemas.time import Time
from sampo.scheduler.topological.base import TopologicalScheduler
from sampo.scheduler.heft.base import HEFTScheduler, HEFTBetweenScheduler

contractors = [ss.contractor(10)]
schedulers = [HEFTScheduler(), HEFTBetweenScheduler(), TopologicalScheduler()]

def generate_result(i: int, wg: WorkGraph) -> list[float]:
    if i % 10 == 0:
        print(f'{i}/{len(graphs)} processed')
    return [int(scheduler.schedule(wg, contractors).execution_time) for scheduler in schedulers]

schedulers_results = [generate_result(i, wg) for i, wg in enumerate(graphs)]
schedulers_results

0/10 processed


[[395, 590, 637],
 [1431, 1440, 1581],
 [1123, 1087, 1166],
 [1208, 1296, 1294],
 [1968, 1863, 1778],
 [1025, 1109, 1156],
 [1389, 1429, 1788],
 [476, 609, 626],
 [1244, 1418, 1333],
 [1239, 1286, 1420]]

In [7]:
def argmin(array) -> int:
    res = 0
    res_v = int(Time.inf())
    for i, v in enumerate(array):
        if v < res_v:
            res_v = v
            res = i
    return res

schedulers_classification = [argmin(result) for result in schedulers_results]
schedulers_classification

[0, 0, 1, 0, 2, 0, 0, 0, 0, 0]

In [8]:
from pandas import DataFrame

df = DataFrame.from_dict(dataset)
df

,0,1,2,3,4
0,36,1,24,1,22
1,150,1,60,1,42
2,129,1,42,1,48
3,131,1,40,1,52
4,198,1,64,1,40
5,129,1,42,1,58
6,139,1,46,1,64
7,39,1,27,1,25
8,141,1,42,1,59
9,135,1,43,1,58


In [9]:
df['label'] = schedulers_classification
df

,0,1,2,3,4,label
0,36,1,24,1,22,0
1,150,1,60,1,42,0
2,129,1,42,1,48,1
3,131,1,40,1,52,0
4,198,1,64,1,40,2
5,129,1,42,1,58,0
6,139,1,46,1,64,0
7,39,1,27,1,25,0
8,141,1,42,1,59,0
9,135,1,43,1,58,0


In [10]:
df.to_csv('dataset.csv')